In [75]:
import pandas as pd
import re
import requests as rq

from selenium import webdriver      as wd
from bs4      import BeautifulSoup  as BS
from tqdm     import tqdm

In [11]:
df = pd.read_csv('Amazon_category.csv')
df.head()

,maincategory,subcategory,category,link
0,0,"tv, audio & cameras",Televisions,https://www.amazon.in/gp/browse.html?node=1389...
1,0,"tv, audio & cameras",Home Entertainment Systems,https://www.amazon.in/gp/browse.html?node=1389...
2,0,"tv, audio & cameras",Headphones,https://www.amazon.in/gp/browse.html?node=1388...
3,0,"tv, audio & cameras",Speakers,https://www.amazon.in/gp/browse.html?node=1389...
4,0,"tv, audio & cameras",Home Audio & Theater,https://www.amazon.in/gp/browse.html?node=1389...


# Generate alternate link

In [12]:
def links(link):
    node_pattern = r'node=(\d+)'
    node_match = re.search(node_pattern, link)
    
    if node_match:
        node_value = node_match.group(1)
        sec_link = f'https://www.amazon.in/s?rh=n%3A{node_value}&fs=true'
        return sec_link
    else:
        return float('nan')
    
df['alt_link'] = df['link'].apply(links)

In [13]:
df.head()

,maincategory,subcategory,category,link,alt_link
0,0,"tv, audio & cameras",Televisions,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389396031&fs=true
1,0,"tv, audio & cameras",Home Entertainment Systems,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389375031&fs=true
2,0,"tv, audio & cameras",Headphones,https://www.amazon.in/gp/browse.html?node=1388...,https://www.amazon.in/s?rh=n%3A1388921031&fs=true
3,0,"tv, audio & cameras",Speakers,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389365031&fs=true
4,0,"tv, audio & cameras",Home Audio & Theater,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389387031&fs=true


In [28]:
for i in df['alt_link']:
    print(i)
    break

https://www.amazon.in/s?rh=n%3A1389396031&fs=true


# Checking status if Alternate link

In [27]:
def iteration_maker(link,driver):
    iteration = 0
    if type(link) is str:
        nxt_link = link
        
        while True:
            
            driver.get(nxt_link)

            content = BS(driver.page_source,'html.parser')
            
            if content: iteration += 1

            tag = 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'
            nxt_link = content.find('a',class_=tag)
            
            if nxt_link:
                nxt_link = 'https://www.amazon.in' + nxt_link.get('href')
    
            else: break
    return iteration

# Counting Iteration

In [38]:
def iteration_count(link):
    iteration = 0
    if type(link) is str:
        nxt_link = link

            
        driver.get(nxt_link)

        content = BS(driver.page_source,'html.parser')
        
        tag = 's-pagination-item s-pagination-disabled'
        
        data = content.find('span',tag)
        
        iteration = int(data.text.strip()) if data else 0
        
        
    return iteration

In [39]:
# driver = wd.Chrome('webdriver/chromedriver.exe')
# _,data = iteration_count('https://www.amazon.in/s?rh=n%3A1389396031&fs=true',driver)

In [40]:
driver = wd.Chrome('webdriver/chromedriver.exe')
df['iteration'] = df['alt_link'].apply(iteration_count)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21100\1340774941.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome('webdriver/chromedriver.exe')


In [41]:
df.head()

,maincategory,subcategory,category,link,alt_link,iteration
0,0,"tv, audio & cameras",Televisions,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389396031&fs=true,46
1,0,"tv, audio & cameras",Home Entertainment Systems,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389375031&fs=true,400
2,0,"tv, audio & cameras",Headphones,https://www.amazon.in/gp/browse.html?node=1388...,https://www.amazon.in/s?rh=n%3A1388921031&fs=true,400
3,0,"tv, audio & cameras",Speakers,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389365031&fs=true,400
4,0,"tv, audio & cameras",Home Audio & Theater,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389387031&fs=true,18


In [43]:
df['iteration'].nunique()

30

In [45]:
df.to_csv('Amazon_link_details.csv',index = False)

In [46]:
df['iteration'].describe()

count    165.000000
mean     289.703030
std      167.464893
min        0.000000
25%       75.000000
50%      400.000000
75%      400.000000
max      400.000000
Name: iteration, dtype: float64

In [47]:
df['iteration'].unique()

array([ 46, 400,  18,   0,  10,  30,  90,  60, 300,  12, 311, 317,  15,
        14, 147,  20,   7,  94,   8, 195, 291,  13,   9,  17, 284, 277,
        75, 172,   6,  56], dtype=int64)

In [49]:
len(df[df['iteration'] < 100])

45

In [61]:
for c,l,i in df[df['iteration'] < 6][['category','alt_link','iteration']].values:
    print(c,l)
    print(i)

DSLR Cameras https://www.amazon.in/s?rh=n%3A1389177031&fs=true
0
Men's Handlooms https://www.amazon.in/s?rh=n%3A11969751031&fs=true
0
Top Brands nan
0
Handloom & Handicraft Store https://www.amazon.in/s?rh=n%3A11971792031&fs=true
0
Sportswear https://www.amazon.in/s?rh=n%3A12302884031&fs=true
0
Explore Showroom nan
0
Shop by Room https://www.amazon.in/s?rh=n%3A10892568031&fs=true
0
Home & Kitchen Deals https://www.amazon.in/s?rh=n%3A4916278031&fs=true
0
Refurbished & Open Box https://www.amazon.in/s?rh=n%3A15290772031&fs=true
0
Editor's Corner https://www.amazon.in/s?rh=n%3A10591963031&fs=true
0
Audible Audiobooks & more https://www.amazon.in/s?rh=n%3A11477925031&fs=true
0
Audible Membership nan
0
Latest Video Games https://www.amazon.in/s?rh=n%3A4069183031&fs=true
0
Open web player nan
0
Amazon Prime Music nan
0


In [60]:
df[df['iteration'] < 6]

,maincategory,subcategory,category,link,alt_link,iteration
6,0,"tv, audio & cameras",DSLR Cameras,https://www.amazon.in/gp/browse.html?node=1389...,https://www.amazon.in/s?rh=n%3A1389177031&fs=true,0
36,1,stores,Men's Handlooms,https://www.amazon.in/gp/browse.html?node=1196...,https://www.amazon.in/s?rh=n%3A11969751031&fs=...,0
42,2,women's clothing,Top Brands,https://www.amazon.in/s?bbn=1953602031&lo=appa...,NaN,0
52,2,stores,Handloom & Handicraft Store,https://www.amazon.in/gp/browse.html?node=1197...,https://www.amazon.in/s?rh=n%3A11971792031&fs=...,0
53,2,stores,Sportswear,https://www.amazon.in/gp/browse.html?node=1230...,https://www.amazon.in/s?rh=n%3A12302884031&fs=...,0
57,3,home & kitchen,Explore Showroom,https://www.amazon.in/showroom?ref_=nav_em_srm...,NaN,0
71,3,home & kitchen,Shop by Room,https://www.amazon.in/gp/browse.html?node=1089...,https://www.amazon.in/s?rh=n%3A10892568031&fs=...,0
72,3,home & kitchen,Home & Kitchen Deals,https://www.amazon.in/gp/browse.html?node=4916...,https://www.amazon.in/s?rh=n%3A4916278031&fs=true,0
100,5,sports & fitness,Refurbished & Open Box,https://www.amazon.in/gp/browse.html?node=1529...,https://www.amazon.in/s?rh=n%3A15290772031&fs=...,0
138,8,books,Editor's Corner,https://www.amazon.in/gp/browse.html?node=1059...,https://www.amazon.in/s?rh=n%3A10591963031&fs=...,0


# Getting qid and ref

In [79]:
dct = {
    'qid':[],
    'ref':[]
}

driver = wd.Chrome('webdriver/chromedriver.exe')
for l,i in tqdm(df[['alt_link','iteration']].values):
    #print(l)
    
    if i != 0:
        driver.get(l)

        content = BS(driver.page_source , 'html.parser')

        tag = 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'
        nxt_link = content.find('a',class_=tag)
        nxt_link = nxt_link.get('href')
        #print('https://www.amazon.in' + nxt_link)
        
        node_pattern = r'qid=(\d+)&ref=([a-zA-Z0-9_]+)'
        node_match = re.search(node_pattern, nxt_link)

        qid = node_match.group(1)
        ref = node_match.group(2)
            #print(qid,ref)
    else:
        #print('None')
        qid = float('nan')
        ref = float('nan')
        
    dct['qid'].append(qid)
    dct['ref'].append(ref)

df_ = pd.DataFrame(dct)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21100\351777777.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome('webdriver/chromedriver.exe')
100%|████████████████████████████████████████████████████████████████████████████████| 165/165 [06:15<00:00,  2.28s/it]


In [80]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     150 non-null    object
 1   ref     150 non-null    object
dtypes: object(2)
memory usage: 2.7+ KB


In [83]:
df_['ref'].unique()

array(['sr_pg_1', nan], dtype=object)

In [81]:
df_.head()

,qid,ref
0,1678609818,sr_pg_1
1,1678609821,sr_pg_1
2,1678609823,sr_pg_1
3,1678609825,sr_pg_1
4,1678609827,sr_pg_1


In [86]:
df = pd.concat((df,df_),axis =1)

In [87]:
df.to_csv('Amazon_full_details.csv',index = False)